In [3]:
import sys
sys.path.append("../src")

In [10]:
from tonpy.types import *
from tonpy.tvm import *
from tonpy.fift.fift import convert_assembler

In [42]:
extension_code = '''
PROGRAM{
  DECLPROC send_msg
  DECLPROC force_chain
  DECLPROC load_data
  DECLPROC store_data
  DECLPROC transfer_ownership
  DECLPROC recv_internal
  102351 DECLMETHOD get_nft_data
  107704 DECLMETHOD get_moderator_address
  91689 DECLMETHOD get_marketplace_address
  122897 DECLMETHOD get_sale_status
  81994 DECLMETHOD get_prev_owner
  82320 DECLMETHOD get_version
  send_msg PROC:<{
    //  to_address amount op query_id payload send_mode
    0 PUSHINT	//  to_address amount op query_id payload send_mode _7=0
    16 PUSHINT	//  to_address amount op query_id payload send_mode _7=0 _8=16
    NEWC	//  to_address amount op query_id payload send_mode _7=0 _8=16 _9
    6 STU	//  to_address amount op query_id payload send_mode _7=0 _11
    s0 s7 XCHG2	//  _7=0 amount op query_id payload send_mode _11 to_address
    STSLICER	//  _7=0 amount op query_id payload send_mode _12
    s0 s5 XCHG2	//  _7=0 send_mode op query_id payload _12 amount
    STVARUINT16	//  _7=0 send_mode op query_id payload _13
    s1 s5 XCHG	//  payload send_mode op query_id _7=0 _13
    107 STU	//  payload send_mode op query_id _27
    s1 s2 XCHG	//  payload send_mode query_id op _27
    32 STU	//  payload send_mode query_id _29
    64 STU	//  payload send_mode msg
    s2 PUSH	//  payload send_mode msg payload
    ISNULL	//  payload send_mode msg _32
    NOT	//  payload send_mode msg _33
    IF:<{	//  payload send_mode msg
      ROT	//  send_mode msg payload
      STBR	//  send_mode msg
      SWAP	//  msg send_mode
    }>ELSE<{
      s2 POP	//  msg send_mode
    }>
    SWAP	//  send_mode msg
    ENDC	//  send_mode _35
    SWAP	//  _35 send_mode
    SENDRAWMSG
  }>
  force_chain PROC:<{
    //  adr
    REWRITESTDADDR	//  _8 _9
    DROP	//  wc
    0 PUSHINT	//  wc _5
    EQUAL	//  _6
    333 THROWIFNOT
  }>
  load_data PROC:<{
    //
    c4 PUSH	//  _1
    CTOS	//  ds
    64 LDU	//  _5 ds
    LDMSGADDR	//  index collection_address ds
    DUP	//  index collection_address ds ds
    SBITS	//  index collection_address ds _10
    0 GTINT	//  index collection_address ds _12
    IFJMP:<{	//  index collection_address ds
      LDMSGADDR	//  index collection_address owner_address ds
      LDREF	//  index collection_address owner_address content ds
      LDREF	//  index collection_address owner_address content _54 _53
      DROP	//  index collection_address owner_address content aditional_data
      CTOS	//  index collection_address owner_address content aditional_data_slice
      LDMSGADDR	//  index collection_address owner_address content marketplace_address aditional_data_slice
      LDMSGADDR	//  index collection_address owner_address content marketplace_address moderator_address aditional_data_slice
      8 LDU	//  index collection_address owner_address content marketplace_address moderator_address sale_status aditional_data_slice
      LDMSGADDR	//  index collection_address owner_address content marketplace_address moderator_address sale_status _62 _61
      DROP	//  index collection_address owner_address content marketplace_address moderator_address sale_status prev_owner
      -1 PUSHINT	//  index collection_address owner_address content marketplace_address moderator_address sale_status prev_owner _37=-1
      8 -ROLL	//  _37=-1 index collection_address owner_address content marketplace_address moderator_address sale_status prev_owner
    }>	//  index collection_address ds
    DROP	//  index collection_address
    0 PUSHINT	//  index collection_address _38=0
    PUSHNULL	//  index collection_address _38=0 _39
    PUSHNULL	//  index collection_address _38=0 _39 _40
    PUSHNULL	//  index collection_address _38=0 _39 _40 _41
    PUSHNULL	//  index collection_address _38=0 _39 _40 _41 _42
    s4 PUSH	//  index collection_address _38=0 _39 _40 _41 _42 _43=0
    s5 s7 XCHG
    s5 s6 XCHG
    PUSHNULL	//  _38=0 index collection_address _39 _40 _41 _42 _43=0 _44
  }>
  store_data PROC:<{
    //  index collection_address owner_address content marketplace_address moderator_address sale_status prev_owner
    NEWC	//  index collection_address owner_address content marketplace_address moderator_address sale_status prev_owner _9
    s0 s4 XCHG2	//  index collection_address owner_address content prev_owner moderator_address sale_status _9 marketplace_address
    STSLICER	//  index collection_address owner_address content prev_owner moderator_address sale_status _10
    ROT	//  index collection_address owner_address content prev_owner sale_status _10 moderator_address
    STSLICER	//  index collection_address owner_address content prev_owner sale_status _11
    8 STU	//  index collection_address owner_address content prev_owner _13
    SWAP	//  index collection_address owner_address content _13 prev_owner
    STSLICER	//  index collection_address owner_address content _14
    ENDC	//  index collection_address owner_address content aditional_data
    s0 s4 XCHG
    NEWC	//  aditional_data collection_address owner_address content index _17
    64 STU	//  aditional_data collection_address owner_address content _19
    s0 s3 XCHG2	//  aditional_data content owner_address _19 collection_address
    STSLICER	//  aditional_data content owner_address _20
    SWAP	//  aditional_data content _20 owner_address
    STSLICER	//  aditional_data content _21
    STREF	//  aditional_data _22
    STREF	//  _23
    ENDC	//  new_data
    c4 POP
  }>
  transfer_ownership PROC:<{
    //  my_balance index collection_address owner_address content sender_address query_id in_msg_body marketplace_address moderator_address prev_owner fwd_fees
    NIP
    s5 POP	//  my_balance index collection_address owner_address content fwd_fees query_id in_msg_body marketplace_address moderator_address
    s0 s2 XCHG	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address in_msg_body
    LDMSGADDR	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address in_msg_body
    OVER	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address in_msg_body new_owner_address
    force_chain CALLDICT
    s1 s2 PUSH2	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address in_msg_body new_owner_address marketplace_address
    SDEQ	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address in_msg_body _16
    IF:<{	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address in_msg_body
      406 THROW
    }>	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address in_msg_body
    LDMSGADDR	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination in_msg_body
    1 LDI	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination _61 _60
    NIP	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination in_msg_body
    LDVARUINT16	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination forward_amount in_msg_body
    60000000 PUSHINT	//  my_balance index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination forward_amount in_msg_body _29
    s1 s13 XCHG	//  in_msg_body index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination forward_amount my_balance _29
    SUB	//  in_msg_body index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination forward_amount _30
    OVER	//  in_msg_body index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination forward_amount _30 forward_amount
    SUB	//  in_msg_body index collection_address owner_address content fwd_fees query_id moderator_address marketplace_address new_owner_address response_destination forward_amount _31
    s0 s7 XCHG	//  in_msg_body index collection_address owner_address content _31 query_id moderator_address marketplace_address new_owner_address response_destination forward_amount fwd_fees
    1 LSHIFT#	//  in_msg_body index collection_address owner_address content _31 query_id moderator_address marketplace_address new_owner_address response_destination forward_amount _33
    s1 s7 XCHG	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination _31 _33
    SUB	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount
    DUP	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount rest_amount
    0 GTINT	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _37
    402 THROWIFNOT
    s6 PUSH	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount forward_amount
    IF:<{	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount
      0x05138d91 PUSHINT	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _39
      NEWC	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _39 _40
      s10 PUSH	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _39 _40 owner_address
      STSLICER	//  in_msg_body index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _39 _41
      s0 s13 XCHG2	//  _39 index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _41 in_msg_body
      STSLICER	//  _39 index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _42
      1 PUSHINT	//  _39 index collection_address owner_address content forward_amount query_id moderator_address marketplace_address new_owner_address response_destination rest_amount _42 _43=1
      s4 PUSH
      s4 s8 XCPU
      s4 s10 XCHG
      s3 s15 XCHG
      s15 s2 XCHG2	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address new_owner_address forward_amount _39 query_id _42 _43=1
      send_msg INLINECALLDICT
    }>ELSE<{
      s6 POP
      s10 POP	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address
    }>
    s9 PUSH	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address response_destination
    2 PLDU	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address _46
    0 NEQINT	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address _48
    IF:<{	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address
      s9 PUSH	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address response_destination
      force_chain CALLDICT
      0xd53276db PUSHINT	//  response_destination index collection_address owner_address content rest_amount query_id moderator_address marketplace_address new_owner_address _50
      s3 s10 XCHG
      s5 s0 s4 XCHG3
      PUSHNULL
      1 PUSHINT	//  moderator_address index collection_address owner_address content marketplace_address new_owner_address response_destination rest_amount _50 query_id _51 _52=1
      send_msg INLINECALLDICT
    }>ELSE<{
      s2 s9 XCHG
      3 2 BLKDROP2	//  moderator_address index collection_address owner_address content marketplace_address new_owner_address
    }>
    s5 s6 XCHG
    s0 s4 XCHG
    s1 s5 s5 XCHG3
    s0 s3 XCHG
    0 PUSHINT
    SWAP	//  index collection_address new_owner_address content marketplace_address moderator_address _54=0 owner_address
    store_data CALLDICT
  }>
  recv_internal PROC:<{
    //  my_balance msg_value in_msg_full in_msg_body
    s2 POP	//  my_balance in_msg_body in_msg_full
    OVER	//  my_balance in_msg_body in_msg_full in_msg_body
    SEMPTY	//  my_balance in_msg_body in_msg_full _4
    IFJMP:<{	//  my_balance in_msg_body in_msg_full
      3 BLKDROP	//
    }>	//  my_balance in_msg_body in_msg_full
    CTOS	//  my_balance in_msg_body cs
    4 LDU	//  my_balance in_msg_body flags cs
    LDMSGADDR	//  my_balance in_msg_body flags sender_address cs
    LDMSGADDR	//  my_balance in_msg_body flags sender_address _248 _247
    NIP	//  my_balance in_msg_body flags sender_address cs
    LDVARUINT16	//  my_balance in_msg_body flags sender_address _250 _249
    NIP	//  my_balance in_msg_body flags sender_address cs
    1 PUSHINT	//  my_balance in_msg_body flags sender_address cs _19=1
    SDSKIPFIRST	//  my_balance in_msg_body flags sender_address cs
    LDVARUINT16	//  my_balance in_msg_body flags sender_address _252 _251
    NIP	//  my_balance in_msg_body flags sender_address cs
    LDVARUINT16	//  my_balance in_msg_body flags sender_address _254 _253
    DROP	//  my_balance in_msg_body flags sender_address fwd_fee
    load_data CALLDICT	//  my_balance in_msg_body flags sender_address fwd_fee init? index collection_address owner_address content marketplace_address moderator_address sale_status prev_owner
    s0 s8 XCHG	//  my_balance in_msg_body flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address sale_status init?
    NOT	//  my_balance in_msg_body flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address sale_status _36
    IFJMP:<{	//  my_balance in_msg_body flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address sale_status
      s7 POP
      4 BLKDROP
      s5 POP	//  my_balance in_msg_body collection_address sender_address fwd_fee sale_status index
      s4 s3 PUXC	//  my_balance in_msg_body collection_address index fwd_fee sale_status collection_address sender_address
      SDEQ	//  my_balance in_msg_body collection_address index fwd_fee sale_status _38
      405 THROWIFNOT
      s0 s4 XCHG	//  my_balance sale_status collection_address index fwd_fee in_msg_body
      LDMSGADDR	//  my_balance sale_status collection_address index fwd_fee owner_address in_msg_body
      OVER	//  my_balance sale_status collection_address index fwd_fee owner_address in_msg_body owner_address
      force_chain CALLDICT
      LDREF	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content in_msg_body
      LDMSGADDR	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address in_msg_body
      OVER	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address in_msg_body marketplace_address
      force_chain CALLDICT
      LDMSGADDR	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body
      OVER	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body moderator_address
      force_chain CALLDICT
      0 PUSHINT	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body _56=0
      NEWC	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body _56=0 _57
      4 STU	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body _59
      ENDC	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body _60
      CTOS	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner
      0 PUSHINT	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner amount=0
      DUP	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner amount=0 op=0
      7235188 PUSHINT	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner amount=0 op=0 query_id=7235188
      PUSHNULL	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner amount=0 op=0 query_id=7235188 body
      s5 PUSH	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner amount=0 op=0 query_id=7235188 body in_msg_body
      SREMPTY	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner amount=0 op=0 query_id=7235188 body _70
      IFNOT:<{	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body prev_owner amount=0 op=0 query_id=7235188 body
        5 BLKDROP	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address in_msg_body
        LDREF	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address _273 _272
        DROP	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address _72
        CTOS	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address extra
        LDMSGADDR	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address _275 _274
        DROP	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address prev_owner
        6000000 PUSHINT	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address prev_owner amount=6000000
        0x05138d91 PUSHINT	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address prev_owner amount=6000000 op
        0 PUSHINT	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address prev_owner amount=6000000 op query_id=0
        NEWC	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address prev_owner amount=6000000 op query_id=0 _80
        s4 PUSH	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address prev_owner amount=6000000 op query_id=0 _80 prev_owner
        STSLICER	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address prev_owner amount=6000000 op query_id=0 body
        4 -ROLL	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address body prev_owner amount=6000000 op query_id=0
      }>ELSE<{
        s5 POP	//  my_balance sale_status collection_address index fwd_fee owner_address nft_body_content marketplace_address moderator_address body prev_owner amount op query_id
      }>
      s8 PUSH
      s7 s11 XCHG
      s6 s12 XCHG
      s0 s5 XCHG
      s4 s8 XCHG
      s3 s11 XCHG
      s12 s13 s8 XCHG3	//  my_balance query_id op amount fwd_fee owner_address body index collection_address owner_address nft_body_content marketplace_address moderator_address sale_status prev_owner
      store_data CALLDICT
      60000000 PUSHINT	//  my_balance query_id op amount fwd_fee owner_address body _84
      s1 s7 XCHG	//  body query_id op amount fwd_fee owner_address my_balance _84
      SUB	//  body query_id op amount fwd_fee owner_address _85
      ROT	//  body query_id op amount owner_address _85 fwd_fee
      SUB	//  body query_id op amount owner_address _86
      s2 PUSH	//  body query_id op amount owner_address _86 amount
      SUB	//  body query_id op amount owner_address rest_amount
      0 GTINT	//  body query_id op amount owner_address _90
      402 THROWIFNOT
      s3 s4 XCHG2
      1 PUSHINT	//  owner_address amount op query_id body _92=1
      send_msg INLINECALLDICT
    }>	//  my_balance in_msg_body flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address sale_status
    s0 s11 XCHG	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address in_msg_body
    32 LDU	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body
    OVER	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body op
    0 EQINT	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _99
    IF:<{	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body
      s10 s5 PUSH2	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body sender_address owner_address
      SDEQ	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _101
      401 THROWIFNOT
      32 LDU	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op command in_msg_body
      SWAP
      1936026732 PUSHINT	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body command _107=1936026732
      EQUAL	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _108
      IF:<{	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body
        s7 s6 s3 PUSH3
        s7 s6 s5 PUSH3
        1 PUSHINT
        s12 PUSH	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body index collection_address marketplace_address content marketplace_address moderator_address _109=1 owner_address
        store_data CALLDICT
        0 PUSHINT	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0
        0x05138d91 PUSHINT	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0 _112
        OVER	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0 _112 _113=0
        NEWC	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0 _112 _113=0 _114
        s9 PUSH	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0 _112 _113=0 _114 owner_address
        STSLICER	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0 _112 _113=0 _115
        s4 PUSH	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0 _112 _113=0 _115 in_msg_body
        STSLICER	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body _111=0 _112 _113=0 _116
        s7 PUSH
        4 -ROLL
        64 PUSHINT	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body marketplace_address _111=0 _112 _113=0 _116 _117=64
        send_msg INLINECALLDICT
        0 THROW
      }>	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body
    }>	//  my_balance sale_status flags sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op in_msg_body
    s0 s11 XCHG
    1 PUSHINT	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op flags _121=1
    AND	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op _122
    IFJMP:<{	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op
      13 BLKDROP	//
    }>	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op
    0xd4ee53fb PUSHINT	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op _123
    s1 s(-1) PUXC	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op op _123
    EQUAL	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op _124
    IFJMP:<{	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op
      s1 s3 XCHG
      3 BLKDROP
      s8 s7 XCHG2
      6 BLKDROP	//  my_balance owner_address moderator_address sender_address
      SWAP	//  my_balance owner_address sender_address moderator_address
      SDEQ	//  my_balance owner_address _126
      401 THROWIFNOT
      60000000 PUSHINT	//  my_balance owner_address _129
      s1 s2 XCHG	//  owner_address my_balance _129
      SUB	//  owner_address _130
      900000 PUSHINT	//  owner_address _130 _131=900000
      SUB	//  owner_address rest_amount
      0 GTINT	//  owner_address _135
      402 THROWIFNOT
      0 PUSHINT	//  owner_address _137=0
      DUP	//  owner_address _137=0 _138=0
      7235188 PUSHINT	//  owner_address _137=0 _138=0 _139=7235188
      PUSHNULL	//  owner_address _137=0 _138=0 _139=7235188 _140
      1 PUSHINT	//  owner_address _137=0 _138=0 _139=7235188 _140 _141=1
      send_msg INLINECALLDICT
    }>	//  my_balance sale_status in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address op
    s0 s10 XCHG	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address in_msg_body
    64 LDU	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
    0x5fcc3d14 PUSHINT	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _147
    s12 s(-1) PUXC	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body op _147
    EQUAL	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _148
    IFJMP:<{	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
      s11 POP
      s11 POP	//  my_balance query_id in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address
      s8 s3 PUSH2	//  my_balance query_id in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address sender_address owner_address
      SDEQ	//  my_balance query_id in_msg_body sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address _150
      401 THROWIFNOT
      s5 s10 XCHG
      s4 s9 XCHG
      s3 s8 XCHG
      s0 s3 s7 XCHG3
      s1 s6 XCHG	//  my_balance index collection_address owner_address content sender_address query_id in_msg_body marketplace_address moderator_address prev_owner fwd_fee
      transfer_ownership INLINECALLDICT
    }>	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
    0xbb684f3e PUSHINT	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _153
    s12 s(-1) PUXC	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body op _153
    EQUAL	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _154
    IFJMP:<{	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
      NIP
      s8 POP
      s9 POP
      s9 POP
      s9 POP	//  content marketplace_address moderator_address sender_address in_msg_body prev_owner index collection_address owner_address
      s1 s5 XCHG	//  content marketplace_address moderator_address collection_address in_msg_body prev_owner index sender_address owner_address
      SDEQ	//  content marketplace_address moderator_address collection_address in_msg_body prev_owner index _156
      401 THROWIFNOT
      s0 s2 XCHG	//  content marketplace_address moderator_address collection_address index prev_owner in_msg_body
      LDMSGADDR	//  content marketplace_address moderator_address collection_address index prev_owner sell_contract_address in_msg_body
      OVER
      s4 s5 XCHG
      s8 s1 s3 XCHG3
      s7 s6 XCHG2
      1 PUSHINT
      s7 PUSH	//  sell_contract_address in_msg_body prev_owner index collection_address sell_contract_address content marketplace_address moderator_address _161=1 prev_owner
      store_data CALLDICT
      0 PUSHINT	//  sell_contract_address in_msg_body prev_owner _163=0
      0x05138d91 PUSHINT	//  sell_contract_address in_msg_body prev_owner _163=0 _164
      OVER	//  sell_contract_address in_msg_body prev_owner _163=0 _164 _165=0
      NEWC	//  sell_contract_address in_msg_body prev_owner _163=0 _164 _165=0 _166
      s0 s4 XCHG2	//  sell_contract_address in_msg_body _165=0 _163=0 _164 _166 prev_owner
      STSLICER	//  sell_contract_address in_msg_body _165=0 _163=0 _164 _167
      s0 s4 XCHG2	//  sell_contract_address _164 _165=0 _163=0 _167 in_msg_body
      STSLICER	//  sell_contract_address _164 _165=0 _163=0 _168
      s3 s3 s0 XCHG3
      64 PUSHINT	//  sell_contract_address _163=0 _164 _165=0 _168 _169=64
      send_msg INLINECALLDICT
    }>	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
    0x2fcb26a2 PUSHINT	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _171
    s12 s(-1) PUXC	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body op _171
    EQUAL	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _172
    IFJMP:<{	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
      s1 s5 XCHG
      5 BLKDROP
      s3 POP
      s3 POP
      s4 POP
      s4 POP
      s4 POP	//  collection_address query_id index sender_address
      0 PUSHINT	//  collection_address query_id index sender_address _173=0
      0x8b771735 PUSHINT	//  collection_address query_id index sender_address _173=0 _174
      s0 s3 XCHG
      NEWC	//  collection_address query_id _174 sender_address _173=0 index _175
      256 STU	//  collection_address query_id _174 sender_address _173=0 _177
      s0 s5 XCHG2	//  _173=0 query_id _174 sender_address _177 collection_address
      STSLICER	//  _173=0 query_id _174 sender_address _178
      s1 s4 XCHG
      s1 s3 XCHG
      64 PUSHINT	//  sender_address _173=0 _174 query_id _178 _179=64
      send_msg INLINECALLDICT
    }>	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
    0xf530d867 PUSHINT	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _181
    s12 s(-1) PUXC	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body op _181
    EQUAL	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body _182
    IFJMP:<{	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
      s3 POP
      s8 POP
      s9 POP
      s10 POP	//  in_msg_body sale_status moderator_address sender_address query_id prev_owner index collection_address owner_address content
      s6 s1 PUSH2	//  in_msg_body sale_status moderator_address sender_address query_id prev_owner index collection_address owner_address content sender_address owner_address
      SDEQ	//  in_msg_body sale_status moderator_address sender_address query_id prev_owner index collection_address owner_address content _184
      401 THROWIFNOT
      s0 s9 XCHG	//  content sale_status moderator_address sender_address query_id prev_owner index collection_address owner_address in_msg_body
      LDMSGADDR	//  content sale_status moderator_address sender_address query_id prev_owner index collection_address owner_address _285 _284
      DROP	//  content sale_status moderator_address sender_address query_id prev_owner index collection_address owner_address new_marketplace_address
      s3 PUSH
      s6 s3 XCPU
      s0 s6 XCHG
      s3 s5 XCHG
      s4 s11 XCHG
      s2 s3 XCHG
      s9 s10 s0 XCHG3	//  collection_address query_id index sender_address index collection_address owner_address content new_marketplace_address moderator_address sale_status prev_owner
      store_data CALLDICT
      0 PUSHINT	//  collection_address query_id index sender_address _190=0
      0xaf573d84 PUSHINT	//  collection_address query_id index sender_address _190=0 _191
      s0 s3 XCHG
      NEWC	//  collection_address query_id _191 sender_address _190=0 index _192
      256 STU	//  collection_address query_id _191 sender_address _190=0 _194
      s0 s5 XCHG2	//  _190=0 query_id _191 sender_address _194 collection_address
      STSLICER	//  _190=0 query_id _191 sender_address _195
      s1 s4 XCHG
      s1 s3 XCHG
      64 PUSHINT	//  sender_address _190=0 _191 query_id _195 _196=64
      send_msg INLINECALLDICT
    }>	//  my_balance sale_status op sender_address fwd_fee prev_owner index collection_address owner_address content marketplace_address moderator_address query_id in_msg_body
    s6 POP	//  my_balance sale_status op sender_address fwd_fee prev_owner index in_msg_body owner_address content marketplace_address moderator_address query_id
    0x7b3520bb PUSHINT	//  my_balance sale_status op sender_address fwd_fee prev_owner index in_msg_body owner_address content marketplace_address moderator_address query_id _198
    s11 s(-1) PUXC	//  my_balance sale_status op sender_address fwd_fee prev_owner index in_msg_body owner_address content marketplace_address moderator_address query_id op _198
    EQUAL	//  my_balance sale_status op sender_address fwd_fee prev_owner index in_msg_body owner_address content marketplace_address moderator_address query_id _199
    IFJMP:<{	//  my_balance sale_status op sender_address fwd_fee prev_owner index in_msg_body owner_address content marketplace_address moderator_address query_id
      DROP
      s7 POP
      s8 POP
      s9 POP	//  content sale_status marketplace_address sender_address moderator_address prev_owner index in_msg_body owner_address
      s5 s4 XCPU	//  content sale_status marketplace_address owner_address moderator_address prev_owner index in_msg_body sender_address moderator_address
      SDEQ	//  content sale_status marketplace_address owner_address moderator_address prev_owner index in_msg_body _201
      401 THROWIFNOT
      LDMSGADDR	//  content sale_status marketplace_address owner_address moderator_address prev_owner index _287 _286
      DROP	//  content sale_status marketplace_address owner_address moderator_address prev_owner index new_collection_address
      s7 s6 XCHG2
      s0 s4 XCHG2
      s0 s5 XCHG
      s3 s1 s3 XCHG3	//  index new_collection_address owner_address content marketplace_address moderator_address sale_status prev_owner
      store_data CALLDICT
    }>	//  my_balance sale_status op sender_address fwd_fee prev_owner index in_msg_body owner_address content marketplace_address moderator_address query_id
    s6 POP
    s6 POP
    2DROP
    s7 POP	//  my_balance owner_address op sender_address fwd_fee moderator_address query_id in_msg_body
    0xd4ee53ff PUSHINT	//  my_balance owner_address op sender_address fwd_fee moderator_address query_id in_msg_body _207
    s6 s(-1) PUXC	//  my_balance owner_address op sender_address fwd_fee moderator_address query_id in_msg_body op _207
    EQUAL	//  my_balance owner_address op sender_address fwd_fee moderator_address query_id in_msg_body _208
    IFJMP:<{	//  my_balance owner_address op sender_address fwd_fee moderator_address query_id in_msg_body
      s5 POP	//  my_balance owner_address in_msg_body sender_address fwd_fee moderator_address query_id
      s3 s3 XCHG2	//  my_balance owner_address in_msg_body query_id fwd_fee sender_address moderator_address
      SDEQ	//  my_balance owner_address in_msg_body query_id fwd_fee _210
      401 THROWIFNOT
      60000000 PUSHINT	//  my_balance owner_address in_msg_body query_id fwd_fee _213
      s1 s5 XCHG	//  fwd_fee owner_address in_msg_body query_id my_balance _213
      SUB	//  fwd_fee owner_address in_msg_body query_id _214
      s0 s4 XCHG	//  _214 owner_address in_msg_body query_id fwd_fee
      1 LSHIFT#	//  _214 owner_address in_msg_body query_id _216
      s1 s4 XCHG	//  query_id owner_address in_msg_body _214 _216
      SUB	//  query_id owner_address in_msg_body rest_amount
      0x05138d91 PUSHINT	//  query_id owner_address in_msg_body rest_amount _218
      NEWC	//  query_id owner_address in_msg_body rest_amount _218 _219
      s4 PUSH	//  query_id owner_address in_msg_body rest_amount _218 _219 owner_address
      STSLICER	//  query_id owner_address in_msg_body rest_amount _218 _220
      s0 s3 XCHG2	//  query_id owner_address _218 rest_amount _220 in_msg_body
      STSLICER	//  query_id owner_address _218 rest_amount _221
      s1 s3 XCHG
      s1 s4 XCHG
      1 PUSHINT	//  owner_address rest_amount _218 query_id _221 _222=1
      send_msg INLINECALLDICT
    }>	//  my_balance owner_address op sender_address fwd_fee moderator_address query_id in_msg_body
    s3 POP
    s5 POP
    s5 POP	//  moderator_address query_id op sender_address in_msg_body
    0x3d6b87c7 PUSHINT	//  moderator_address query_id op sender_address in_msg_body _224
    s1 s3 XCHG	//  moderator_address query_id in_msg_body sender_address op _224
    EQUAL	//  moderator_address query_id in_msg_body sender_address _225
    IFJMP:<{	//  moderator_address query_id in_msg_body sender_address
      s0 s3 PUXC	//  sender_address query_id in_msg_body sender_address moderator_address
      SDEQ	//  sender_address query_id in_msg_body _227
      401 THROWIFNOT
      LDREF	//  sender_address query_id _289 _288
      DROP	//  sender_address query_id new_code
      DUP	//  sender_address query_id new_code new_code
      SETCODE
      CTOS	//  sender_address query_id _233
      BLESS	//  sender_address query_id _234
      c3 POP
      0 PUSHINT	//  sender_address query_id _236=0
      0x5cbb3a8e PUSHINT	//  sender_address query_id _236=0 _237
      ROT
      PUSHNULL
      64 PUSHINT	//  sender_address _236=0 _237 query_id _238 _239=64
      send_msg INLINECALLDICT
    }>	//  moderator_address query_id in_msg_body sender_address
    4 BLKDROP	//
    16 PUSHPOW2DEC	//  _241=65535
    THROWANY
  }>
  get_nft_data PROC:<{
    //
    load_data CALLDICT	//  _10 _11 _12 _13 _14 _15 _16 _17 _18
    4 BLKDROP	//  init? index collection_address owner_address content
  }>
  get_moderator_address PROC:<{
    //
    load_data CALLDICT	//  _10 _11 _12 _13 _14 _15 _16 _17 _18
    s2 s8 XCHG
    8 BLKDROP	//  moderator_address
  }>
  get_marketplace_address PROC:<{
    //
    load_data CALLDICT	//  _10 _11 _12 _13 _14 _15 _16 _17 _18
    s3 s8 XCHG
    8 BLKDROP	//  marketplace_address
  }>
  get_sale_status PROC:<{
    //
    load_data CALLDICT	//  _10 _11 _12 _13 _14 _15 _16 _17 _18
    s1 s8 XCHG
    8 BLKDROP	//  sale_status
  }>
  get_prev_owner PROC:<{
    //
    load_data CALLDICT	//  _10 _11 _12 _13 _14 _15 _16 _17 _18
    8 1 BLKDROP2	//  prev_owner
  }>
  get_version PROC:<{
    //
    16 PUSHINT	//  _0=12
  }>
}END>c
'''


In [43]:
f = Fift()

In [44]:
f.run(code)

RuntimeError: top: "
level 1: <continuation 0x600001b8a100>
level 2: <text interpreter continuation>
